---

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import random
from IPython import display
from matplotlib.animation import FuncAnimation

# Diffusion-Limited Aggregation

Brownian motion is the motion of a particle, such as a smoke or dust particle, in a gas, as it is buffeted by random collisions with gas molecules. The first part of this simulation is of such a particle in two dimensions as follows. In general, the particle is confined to a square grid or lattice $L\times L$ squares on a side, so that its position can be represented by two integers. It starts in the middle of the grid. On each step of the simulation, a random direction - up, down, left, or right - is chosen, and the particle is moved one step in that direction, i.e. a random walk.

For this demonstration, 100 steps of this process on a lattice with $L=25$ will be performed, and an animation will plot the position of the particle at each step. If the particle hits a wall, it will bounce off the wall in the opposite direction.

In [10]:
fig = plt.figure()
pbar = display.ProgressBar(100)
pbar.display()
    
lines = plt.plot(0, 0, 'o')     # starting in the center
point = lines[0]

plt.axis("scaled")
plt.xlim(-12, 12)    # for a 25x25 grid with 0,0 at the center, the max coordinate for either x or y is 12
plt.ylim(-12, 12)

plt.grid(linewidth = 1)

def animate(frame):
    x = point.get_xdata()
    y = point.get_ydata()
    # generate a number from 1-4 to represent moving in one of the four possible directions
    next_step = random.randint(1, 4)
    
    if(next_step == 1):
        if(point.get_ydata() == 12):
            # bounce off
            point.set_ydata(y - 1)
        else:
            # move up
            point.set_ydata(y + 1)
    elif(next_step == 2):
        if(point.get_xdata() == 12):
            # bounce off
            point.set_xdata(x - 1)
        else:
            # move right
            point.set_xdata(x + 1)
    elif(next_step == 3):
        if(point.get_ydata() == -12):
            # bounce off
            point.set_ydata(y + 1)
        else:
            # move down
            point.set_ydata(y - 1)
    elif(next_step == 4):
        if(point.get_xdata() == -12):
            # bounce off
            point.set_xdata(x + 1)
        else:
            # move left
            point.set_xdata(x - 1)
    
    pbar.progress = frame+1

anim = FuncAnimation(fig, animate, frames=100, interval=100)
video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()

[============================================================] 100/100

Next, I will be reproducing one a famous model in computational physics, [diffusion-limited aggregation](https://en.wikipedia.org/wiki/Diffusion-limited_aggregation), or DLA for short. There are various versions of DLA, but the one in this program is as follows. Take a square grid with a single particle in the middle. The particle performs a random walk from square to square on the grid until it reaches a point on the edge of the system, at which point it "sticks" to the edge, becoming anchored there and immovable. Then a second particle starts at the center and does a random walk until it
sticks either to an edge or to the other particle. Then a third particle starts, and so on. Each particle starts at the center and walks until it sticks either to an edge or to any anchored particle.


The general concept of the walk in this case is very similar to the previous bit of code for the random walk. This time, a $101 \times 101$ grid will be used. New particles will be created at the center of the grid, whereupon they will walk randomly until they stick to either an edge or another anchored particle.


For the sake of speed (both of runtime and the animation itself), the particles will only be shown on screen once they are anchored. The entire path of each random walk will not be animated.

Set up the program so that it generates a total of 1000 particles (corresponding to 1000 frames of animation).

In [8]:
# Represent the border and any anchored particles with a boolean (i.e. 1 or 0) matrix.
# Initialize to False, then make all of the borders True so that particles
# will anchor to the border by default.
fixed_points = np.zeros((101, 101), bool)
fixed_points[0] = np.ones(101, bool)
fixed_points[100] = np.ones(101, bool)
fixed_points[:, 0] = np.ones((1, 101), bool)
fixed_points[:, 100] = np.ones((1, 101), bool)

fig = plt.figure()
pbar = display.ProgressBar(1000)
pbar.display()

plt.axis("scaled")
plt.xlim(-50, 50)
plt.ylim(-50, 50)
plt.grid(linewidth = 1)

def needs_anchor(x, y):
    """
    Determines if any of the neighboring spots are occupied.
    If so, sum will be non-zero, meaning the logical negation of it 
    will break out of the while loop.
    Input: indices of matrix corresponding to xy coordinate of current particle
    """
    return fixed_points[x][y-1] + fixed_points[x+1][y] + fixed_points[x][y+1] + fixed_points[x-1][y]


"""
Matrix elements are indexed starting at the top left corner, whereas our grid has (0, 0) in the center.
Thus, indexing the matrix in the following way will map our particle's coordinates to the appropriate indices.
With coordinates, it's (column, row), i.e. (x, y),
but with matrix indexing, it's [row][column], which is equivalent to [y][x]
e.g. (-50, 50) -> [0][0], (-50, -50) -> [100][0], (50, 50) -> [0][100]
"""
def calculate_path(point):
    # Data getters for Line2d object return lists. 
    # Index it to just get the integer typed data,
    # which in this case is the only element of each list.
    x = point.get_xdata()[0]
    y = point.get_ydata()[0]

    # See above paragraph to explain offsets of 50
    while(not needs_anchor(x+50, 50-y)):    
        next_step = random.randint(1, 4)
        if(next_step == 1):
            # move up
            y += 1
        elif(next_step == 2):
            # move right
            x += 1
        elif(next_step == 3):
            # move down
            y -= 1
        elif(next_step == 4):
            # move left
            x -= 1
    
    # by now, the particle has encountered either a wall or another particle,
    # so fix it in place
    fixed_points[50-y][x+50] = True
    return (x, y)
                                            
def animate(frame):
    lines = plt.plot(0, 0, 'o')
    point = lines[0]
    final_x, final_y = calculate_path(point)
    # just plot final position rather than the path, to save on runtime and animation length
    point.set_data((final_x, final_y))
    pbar.progress = frame+1

# 1000 frames ==> 1000 particles/random walks generated
anim = FuncAnimation(fig, animate, frames=1000, interval=20)
video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()

[============================================================] 1000/1000